# Analysis of the first round of the 2016 Bulgarian presidential elections

## Part II - Pre-processing


In Part I we scraped data about the number of ballots cast abroad for each of the top 6 candidates in the first round of Bulgaria's presidential elections. 

In Part II below we will examine the data for inconsistencies and incomplete entries, and calculate variables of interest. 

In Part III we will visually examine the data for any interesting patterns. 

### Step 1. Review the data

In [1]:
# Load the data and examine its structure
setwd("C:/Users/grozeve/Documents/_9_Misc/DataScience/My projects/bg_election")
ballots.df <- read.csv("ballots_english.csv", stringsAsFactor=FALSE)
head(ballots.df)

,X,section.id,country,city,country_c,city_c,valid,protest,tt,rr,cc,kk,vm,po
1,1,320100003,Australia,Brisbane,<U+0410><U+0432><U+0441><U+0442><U+0440><U+0430><U+043B><U+0438><U+044F>,<U+0411><U+0440><U+0438><U+0437><U+0431><U+0435>,45,2,14,13,4,7,4,0
2,2,320100001,Australia,Canberra,<U+0410><U+0432><U+0441><U+0442><U+0440><U+0430><U+043B><U+0438><U+044F>,<U+041A><U+0430><U+043D><U+0431><U+0435><U+0440>,40,1,3,17,11,0,1,0
3,3,320100004,Australia,Melbourne,<U+0410><U+0432><U+0441><U+0442><U+0440><U+0430><U+043B><U+0438><U+044F>,<U+041C><U+0435><U+043B><U+0431><U+044A><U+0440>,115,3,45,13,24,22,1,0
4,4,320100005,Australia,Perth,<U+0410><U+0432><U+0441><U+0442><U+0440><U+0430><U+043B><U+0438><U+044F>,<U+041F><U+044A><U+0440><U+0442>,81,9,30,11,11,7,2,0
5,5,320100002,Australia,Sidney,<U+0410><U+0432><U+0441><U+0442><U+0440><U+0430><U+043B><U+0438><U+044F>,<U+0421><U+0438><U+0434><U+043D><U+0438>,148,8,60,22,16,10,8,1
6,6,320200012,Austria,Bregenz,<U+0410><U+0432><U+0441><U+0442><U+0440><U+0438><U+044F>,<U+0411><U+0440><U+0435><U+0433><U+0435><U+043D><U+0446>,122,13,23,19,24,14,19,0


In [2]:
# Remove the country and city names in Cyrillic
ballots.df <- ballots.df[,c(2,3,4,7:14)]

In [5]:
# Examine each variable
str(ballots.df)
summary(ballots.df)

'data.frame':	325 obs. of  11 variables:
 $ section.id: int  320100003 320100001 320100004 320100005 320100002 320200012 320200006 320200007 320200013 320200302 ...
 $ country   : chr  "Australia" "Australia" "Australia" "Australia" ...
 $ city      : chr  "Brisbane" "Canberra" "Melbourne" "Perth" ...
 $ valid     : int  45 40 115 81 148 122 608 761 558 505 ...
 $ protest   : int  2 1 3 9 8 13 32 61 35 30 ...
 $ tt        : int  14 3 45 30 60 23 149 243 177 154 ...
 $ rr        : int  13 17 13 11 22 19 100 141 112 89 ...
 $ cc        : int  4 11 24 11 16 24 112 107 75 77 ...
 $ kk        : int  7 0 22 7 10 14 71 82 66 72 ...
 $ vm        : int  4 1 1 2 8 19 51 43 28 28 ...
 $ po        : int  0 0 0 0 1 0 20 11 4 7 ...


   section.id          country              city               valid       
 Min.   :320100001   Length:325         Length:325         Min.   :   0.0  
 1st Qu.:321300063   Class :character   Class :character   1st Qu.: 124.0  
 Median :323000143   Mode  :character   Mode  :character   Median : 229.0  
 Mean   :323567548                                         Mean   : 339.5  
 3rd Qu.:325800224                                         3rd Qu.: 481.0  
 Max.   :327200301                                         Max.   :2024.0  
    protest            tt              rr               cc        
 Min.   : 0.00   Min.   :  0.0   Min.   :  0.00   Min.   :  0.00  
 1st Qu.: 5.00   1st Qu.:  9.0   1st Qu.: 18.00   1st Qu.: 20.00  
 Median :14.00   Median : 27.0   Median : 40.00   Median : 43.00  
 Mean   :19.14   Mean   : 44.8   Mean   : 55.81   Mean   : 84.78  
 3rd Qu.:30.00   3rd Qu.: 56.0   3rd Qu.: 82.00   3rd Qu.: 92.00  
 Max.   :96.00   Max.   :326.0   Max.   :279.00   Max.   :847.00  

Note that the minimum number of valid votes in the dataframe is 0. Therefore, we will identify which sections had 0 valid votes.

In [7]:
ballots.df[which(ballots.df$valid==0), ] 

,section.id,country,city,valid,protest,tt,rr,cc,kk,vm,po
110,322300105,Spain,Aranda de Duaro,0,0,5,65,40,43,16,8


Since there is a positive number of ballots cast for the individual candidates, the data for section.id=322300105 is inconsistently input in the source. We will correct the number of valid votes manually. 

Note also that while this data point seemed like an outlier to be discarded, it was simply the result of an inconsistency error, which can be corrected. In addition, if we had captured data for all candidates on the ballot list, we could have checked for internal consistency by adding up all the protest and candidate votes and verifying that they match the number of valid votes.

In [8]:
# Insert the correct number of valid ballots into the dataframe for section 322300105
# We calculate the correct number of valid ballots using information from the section page
ballots.df[which(ballots.df$valid==0), 4] <- 1000 - 765 - 1 -5 

# Inspect the result
ballots.df[which(ballots.df$section.id==322300105), ]

,section.id,country,city,valid,protest,tt,rr,cc,kk,vm,po
110,322300105,Spain,Aranda de Duaro,229,0,5,65,40,43,16,8


Next, let's check for any NAs and run more inconsistency checks.

In [9]:
apply(ballots.df, 2, function(x) any(is.na(x)))

section.id    country       city      valid    protest         tt         rr 
     FALSE      FALSE      FALSE      FALSE      FALSE      FALSE      FALSE 
        cc         kk         vm         po 
     FALSE      FALSE      FALSE      FALSE

In [12]:
ballots.df[which(ballots.df$valid<ballots.df$protest), ]
ballots.df[which(ballots.df$valid<rowSums(ballots.df[,c(6:11)])), ]

,section.id,country,city,valid,protest,tt,rr,cc,kk,vm,po


,section.id,country,city,valid,protest,tt,rr,cc,kk,vm,po


### Step 2. Create variables

We will create the following variables for each voting section:
* Number of valid votes net of protest votes 
* Percent of votes for each candidate as a share of net votes
* Share of the total votes that the top 6 candidates captured
* Rank of each candidate by number of votes

In [13]:
# Calculate number of valid ballots net of protest votes
ballots.df$valid.net <- ballots.df$valid - ballots.df$protest

In [14]:
# Calculate percentage values for each candidate 
ballots.df$tt.perc <- ballots.df$tt/ballots.df$valid.net
ballots.df$rr.perc <- ballots.df$rr/ballots.df$valid.net
ballots.df$cc.perc <- ballots.df$cc/ballots.df$valid.net
ballots.df$kk.perc <- ballots.df$kk/ballots.df$valid.net
ballots.df$vm.perc <- ballots.df$vm/ballots.df$valid.net
ballots.df$po.perc <- ballots.df$po/ballots.df$valid.net

In [15]:
# Calculate what share of the total votes these six candidates (Top 6) captured
ballots.df$top6.ballots <- rowSums(ballots.df[ ,c(6:11)])
ballots.df$top6.perc  <- ballots.df$top6.ballots/ballots.df$valid
summary(ballots.df$top6.perc)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.5789  0.7905  0.8239  0.8293  0.8660  0.9908 

These 6 candidates capture at least 58% of the total ballots in each section and 83% on average. We can further examine how many sections voted for candidates outside of the top 6 that accounted for up to 30% of the vote. 

In [18]:
# Number of sections where the top 6 candidates collected less than 70% of the vote
nrow(ballots.df[which(ballots.df$top6.perc < .70), ])

[1] 13

So, out of the 325 voting sections, in 312 of them the top 6 candidates garnered at least 70% of the vote.

Next, we create rank variables.

In [20]:
ranks <- t(apply(ballots.df[ ,c(6:11)], 1, function(x) rank(-x, ties.method="min")))
ballots.df <- cbind(ballots.df, ranks)
colnames(ballots.df)[c(21:26)] <- c("tt.rank", "rr.rank", "cc.rank", "kk.rank", "vm.rank", "po.rank")

Review the data we added.

In [24]:
head(ballots.df[ ,c(1:3, 12:26)]) 

,section.id,country,city,valid.net,tt.perc,rr.perc,cc.perc,kk.perc,vm.perc,po.perc,top6.ballots,top6.perc,tt.rank,rr.rank,cc.rank,kk.rank,vm.rank,po.rank
1,320100003,Australia,Brisbane,43,0.325581395348837,0.302325581395349,0.0930232558139535,0.162790697674419,0.0930232558139535,0,42,0.933333333333333,1,2,4,3,4,6
2,320100001,Australia,Canberra,39,0.0769230769230769,0.435897435897436,0.282051282051282,0,0.0256410256410256,0,32,0.8,3,1,2,5,4,5
3,320100004,Australia,Melbourne,112,0.401785714285714,0.116071428571429,0.214285714285714,0.196428571428571,0.00892857142857143,0,105,0.91304347826087,1,4,2,3,5,6
4,320100005,Australia,Perth,72,0.416666666666667,0.152777777777778,0.152777777777778,0.0972222222222222,0.0277777777777778,0,61,0.753086419753086,1,2,2,4,5,6
5,320100002,Australia,Sidney,140,0.428571428571429,0.157142857142857,0.114285714285714,0.0714285714285714,0.0571428571428571,0.00714285714285714,117,0.790540540540541,1,2,3,4,5,6
6,320200012,Austria,Bregenz,109,0.211009174311927,0.174311926605505,0.220183486238532,0.128440366972477,0.174311926605505,0,99,0.811475409836066,2,3,1,5,3,6


Now that we have added the variables of interest, we will save the dataframe to a .csv file in order to load the data in Part III, when we will analyze the data.

In [25]:
write.csv(ballots.df, "ballots.csv")